In [2]:
#import needed libraries
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

#%matplotlib inline
import numpy as np
#import matplotlib.pyplot as plt
from sklearn import cross_validation
import tensorflow as tf
import random
from keras.utils import np_utils


from keras.layers.convolutional import ZeroPadding2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
#import data
trainData=np.genfromtxt('train.csv',delimiter=',')
trainingPixmaps=trainData[:,9:438]/255
trainingClasses=trainData[:,1]

In [4]:
#import test data for printing
testData=np.genfromtxt('test.csv',delimiter=',')
testPixmaps = testData[:,9:438]/255
testIds = testData[:,0]

In [127]:
img2 = testPixmaps[105].reshape(33,13)

#smallImage = smallImage[10:24,1:13]
plt.imshow(img2, 'gray')
#plt.imshow(scaledImage, 'gray')
plt.show()

In [66]:
#doing cross validation
x_train, x_test, y_train, y_test = cross_validation.train_test_split (trainingPixmaps, 
                                                                      trainingClasses,    
                                                                      test_size=0.25, 
                                                                      random_state=0)

In [17]:
#change the input format for y train and y test
def changeY(ys) :
    newYs = []
    for i in range (0,len(ys)):
        index = int(ys[i])
        newPoint = np.zeros(98)
        newPoint[index] = 1
        newYs.append(newPoint)
    return np.asarray(newYs)

In [18]:
#change the format
Y_train = changeY(y_train)
Y_test = changeY(y_test)

In [19]:
#reshape the format of x train and x test
X_train = x_train.reshape(x_train.shape[0], 13, 33,1)
X_test = x_test.reshape(x_test.shape[0], 13, 33, 1)

In [20]:
#define the data type for x train and x test
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [32]:
#do CNN with 2 convolutional layers, 2 pooling layers, 2 dense layer, 
#1 activation layer
model = Sequential()


#layer 1
model.add(Convolution2D(18, 5, 5,input_shape = (13,33,1),border_mode='same'))
model.add(Activation('relu'))

#pooling 
model.add(MaxPooling2D(pool_size=(2,2)))

#layer 2
ZeroPadding2D((1,1))
model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))

#pool
model.add(MaxPooling2D(pool_size=(2,2)))


#drop out with dense
model.add(Flatten())
model.add(Dense(2304,init = 'he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.5))


#dense with dropout
model.add(Dense(2500,init = 'he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#softmax layer

model.add(Dense(98,init = 'he_normal'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [33]:
#generate accuracy based on cross validation
model.fit(X_train, Y_train, batch_size=30, nb_epoch=10,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 37711 samples, validate on 12571 samples
Epoch 1/10
37711/37711 [==============================] - 268s - loss: 2.1094 - acc: 0.5268 - val_loss: 1.7842 - val_acc: 0.6291
Epoch 2/10
37711/37711 [==============================] - 264s - loss: 1.7315 - acc: 0.6449 - val_loss: 1.6742 - val_acc: 0.6599
Epoch 3/10
37711/37711 [==============================] - 263s - loss: 1.6523 - acc: 0.6639 - val_loss: 1.6745 - val_acc: 0.6637
Epoch 4/10
37711/37711 [==============================] - 265s - loss: 1.5973 - acc: 0.6733 - val_loss: 1.6792 - val_acc: 0.6644
Epoch 5/10
37711/37711 [==============================] - 270s - loss: 1.5569 - acc: 0.6789 - val_loss: 1.6354 - val_acc: 0.6715
Epoch 6/10
37711/37711 [==============================] - 256s - loss: 1.5180 - acc: 0.6846 - val_loss: 1.6933 - val_acc: 0.6740
Epoch 7/10
37711/37711 [==============================] - 258s - loss: 1.4857 - acc: 0.6894 - val_loss: 1.6265 - val_acc: 0.6758
Epoch 8/10
37711/37711 [=======================

In [34]:
#make the prediction. put predict data into a list
np.genfromtxt('test.csv',delimiter=',')

theTestData = testData[:,9:438]
theTestData = theTestData.reshape(theTestData.shape[0], 13, 33, 1)

predClasses = model.predict( theTestData, batch_size=5, verbose=0)

testIds = testData[:,0]

toSubmit = []
for i in range(0, len(testData)) :
    temp = [testIds[i]]
    temp.append(np.argmax(predClasses[i]))
    toSubmit.append(temp)

In [35]:
#print out the list
import csv 

with open("KerasCNN.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(toSubmit)